In [1]:
import warnings; warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
import nltk
import string
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [3]:
from pathlib import Path
import matplotlib.pyplot as plt

In [4]:
import numpy as np
import pandas as pd

In [5]:
path = Path('C:/Users/sappusamy/Documents/SriWK/datasets/SPAM.csv')

In [6]:
dataset = pd.read_csv(path)

In [7]:
dataset.columns

Index(['Category', 'Message'], dtype='object')

In [8]:
dataset.Category.value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [9]:
ham = dataset[dataset.Category=='ham']

In [10]:
spam = dataset[dataset.Category=='spam']

In [11]:
train = ham[:50]
train = train.append(spam[:50])
train = train.sample(frac=1,random_state=42).reset_index(drop=True)

In [12]:
test = ham[50:100]
test = test.append(spam[50:100])
test = test.sample(frac=1,random_state=42).reset_index(drop=True)

## NLP: Cleaning & Preprocessing text
- Remove HTML
- Tokenization + Remove punctuation
- Remove stopwords
- Lemmatization or stemming

In [13]:
text = train.Message[0]

In [14]:
stop_words = stopwords.words('english')

In [15]:
def remove_html(text):
    return BeautifulSoup(text,'lxml').get_text()

In [16]:
def remove_punctuation(text):
    return "".join([c for c in text if c not in string.punctuation])

In [17]:
def remove_stopwords(text):
    return [w for w in text if w not in stop_words]

In [18]:
def word_lemmatizer(text):
    return [lemmatizer.lemmatize(i) for i in text]

In [19]:
def word_stemmer(text):
    return [stemmer.stem(i) for i in text]

In [20]:
tokenizer = RegexpTokenizer(r'\w+')

In [21]:
lemmatizer = WordNetLemmatizer()

In [22]:
stemmer = PorterStemmer()

In [23]:
t = train['Message'].apply(remove_html)

t = t.apply(remove_punctuation)

t = t.apply(lambda x: tokenizer.tokenize(x.lower()))

t = t.apply(remove_stopwords)

t = t.apply(word_lemmatizer)

t = t.apply(word_stemmer)

In [24]:
labels = train['Category']

set(labels)

{'ham', 'spam'}

In [25]:
labels_index = {'ham':0,'spam':1}

#### Represent each word in ONE-HOT encoding
- as of now, for reducing vector length of input we set min frequency of token to be **SOME INT VALUE**

In [26]:
def token_frequency(l):
    count = {}
    for tokens in l:
        for token in tokens:
            if token not in count:
                count[token]=1
            else:
                count[token]+=1
    return count

In [27]:
tf = token_frequency(t)

In [28]:
sorted(tf.items(),key=lambda x:x[1],reverse=True)[:10]

[('u', 23),
 ('call', 21),
 ('2', 17),
 ('go', 16),
 ('free', 16),
 ('ur', 14),
 ('mobil', 13),
 ('text', 12),
 ('txt', 11),
 ('claim', 11)]

In [29]:
min_freq = 1

In [30]:
def build_vocabulary(tf):
    v = {}
    v['pad']=0
    v['unk']=1
    for token in tf:
        if tf[token]>=min_freq:
            v[token] = len(v)
    return v

In [31]:
vocab = build_vocabulary(tf)

In [32]:
len(vocab)

749

## Let's convert tokens to index and make vector for each word in a review

In [33]:
tokens = t[0]
tokens[:10],len(tokens)

(['500',
  'new',
  'mobil',
  '2004',
  'must',
  'go',
  'txt',
  'nokia',
  '89545',
  'collect'],
 15)

In [34]:
indices = [vocab[token] if token in vocab else vocab['unk'] for token in tokens]
len(indices)

15

## Let's convert these indices to one-hot vectors
- shape of input will be sequence_length * vocab_size

In [35]:
x = torch.zeros(len(indices),len(vocab))

In [36]:
x[range(len(indices)),indices]=1

In [37]:
x,x.shape

(tensor([[0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]), torch.Size([15, 749]))

## Equation of RNN:
\begin{equation*}
h_t = tanh( W_{ih}X_t + b_{ih} + W_{hh}h_{t-1} + b_{hh})
\end{equation*}

In [38]:
rnn_cell = nn.RNNCell(len(vocab),100)

In [39]:
for name,p in rnn_cell.named_parameters():
    print(name,p.shape)

weight_ih torch.Size([100, 749])
weight_hh torch.Size([100, 100])
bias_ih torch.Size([100])
bias_hh torch.Size([100])


In [40]:
h = torch.zeros(1,100)

In [41]:
h

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])

In [42]:
for i in range(len(indices)):
    h = rnn_cell(x[i].view(1,-1),h)

In [43]:
print("**value of hidden vector after iterating over all input time steps**")
h

**value of hidden vector after iterating over all input time steps**


tensor([[ 4.1341e-02,  2.8719e-02, -1.0772e-02,  6.0397e-02,  9.1936e-05,
         -1.8411e-02,  1.4357e-02, -2.1835e-01,  1.2941e-01, -2.9292e-01,
         -1.3177e-01,  1.2926e-03, -4.9819e-03, -2.2580e-01,  8.8884e-03,
          3.2912e-02, -1.1327e-01, -5.6633e-02,  1.8505e-01, -9.2389e-02,
         -8.8952e-02, -8.9404e-02,  1.3897e-02,  8.5317e-02,  6.4917e-02,
         -1.9610e-01, -6.6982e-02, -8.1507e-02, -1.3211e-02,  2.4464e-01,
         -2.5653e-01, -6.1257e-02,  2.0731e-01,  2.4796e-01,  9.5485e-02,
         -1.2610e-01,  8.1142e-02, -4.1523e-02,  2.9730e-02, -9.0939e-02,
         -2.2146e-01, -1.0134e-01,  1.0169e-01,  1.4718e-01, -1.4418e-01,
         -3.7013e-02, -5.1221e-02, -9.5732e-02,  4.8320e-03, -7.1987e-02,
         -1.8768e-02, -4.5836e-02,  1.2190e-01,  1.1848e-01,  5.8956e-02,
          2.0251e-01,  1.9261e-01, -2.2986e-01,  1.2078e-01, -9.9152e-02,
         -7.9297e-02,  1.3925e-01, -8.1363e-02,  6.8298e-02, -2.3147e-02,
         -3.3784e-01,  9.5200e-02,  6.

In RNNCell,
    
    inputs are looped over each time steps

<a href="https://pytorch.org/docs/stable/nn.html?highlight=rnncell#torch.nn.RNNCell">PyTorch RNNCell reference</a>

## RNN for 1 training example

In [44]:
rnn = nn.RNN(len(vocab),100,batch_first=True)

In [45]:
indices = [vocab[token] if token in vocab else vocab['unk'] for token in tokens]

In [46]:
x = torch.zeros(len(indices),len(vocab))

x[range(len(indices)),indices]=1


In [47]:
x = x.expand((1,-1,len(vocab)))
x,x.shape

(tensor([[[0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]), torch.Size([1, 15, 749]))

**Shape of h:-**
    
    h = (num_layers*num_directions, bacth_size, hidden_units)
    
    where num_layers parameter take int value as inputs which build ups STACKED RNN

In [48]:
h = torch.zeros(1,1,100)

In [49]:
output,hidden = rnn(x,h)

In [50]:
output.shape,hidden.shape

(torch.Size([1, 15, 100]), torch.Size([1, 1, 100]))

**output** - output of all time steps of all batches

**hidden** - output of last time step of all batches

In [51]:
classifier = nn.Linear(100,2)

In [52]:
preds = classifier(hidden.view(1,100))
preds

tensor([[-0.0133, -0.0085]], grad_fn=<AddmmBackward>)

## RNN for m training examples

In [53]:
indices = [[vocab[token] if token in vocab else vocab['unk'] for token in doc] for doc in t]

In [54]:
crit = nn.CrossEntropyLoss()

In [55]:
optimizer = optim.SGD([{'params':rnn.parameters()},{'params':classifier.parameters()}],lr=0.01)

In [56]:
for doc,y in zip(indices,labels):
    x = torch.zeros(len(doc),len(vocab))
    x[range(len(doc)),doc]=1
    x = x.expand((1,-1,len(vocab)))
    
    optimizer.zero_grad()
    
    h = torch.zeros(1,1,100)
    output,hidden = rnn(x,h)
    preds = classifier(hidden.view(1,100))
    
    loss = crit(preds,torch.LongTensor([labels_index[y]]))
    loss.backward()
    
    optimizer.step()
    
    print(loss.item())
    

0.6907348036766052
0.7733471989631653
0.6941800117492676
0.6946882605552673
0.6787651777267456
0.6782664060592651
0.6580341458320618
0.7723941206932068
0.6276066899299622
0.6285117864608765
0.6459813117980957
0.5659758448600769
0.770775318145752
0.6630669832229614
0.7781795263290405
0.5748857259750366
0.8560469150543213
0.7791211009025574
0.666670024394989
0.6679280996322632
0.8450742959976196
0.7599931955337524
0.6608483791351318
0.6080837845802307
0.7753790616989136
0.6121601462364197
0.5934309959411621
0.8441199064254761
0.6034643054008484
0.809683084487915
0.6284765601158142
0.6458376049995422
0.8057158589363098
0.5511355400085449
0.7776252031326294
0.6398602724075317
0.7940396070480347
0.7654731273651123
0.6363545060157776
0.6081461906433105
0.598836362361908
0.6256642937660217
0.5477328300476074
0.8283072710037231
0.5811066627502441
0.5175595283508301
0.8443266749382019
0.5660105347633362
0.8884944915771484
0.6143222451210022
0.5051776766777039
0.5066477060317993
0.54124784469604

**Why loop over each document?**

    different documents are of differnt lengths

## Batch-wise processing

**Why do we need padding?**
- sequence length of different documents will be of different length
- so flow will be as follows:<br>
      for doc in documents:
          for token in doc:
              pass to RNNCell
- the above methods has huge complexity
- in order to process in batch-wise, we make all documents length to be of same length

    
    Thus padding comes in, which means pads 0's to documents which has length less than max length

In [57]:
indices = [[vocab[token] if token in vocab else vocab['unk'] for token in doc] for doc in t]

In [58]:
sequence_lengths = [len(d) for d in indices]

In [59]:
max_length = len(max(t,key=lambda x:len(x)))
max_length

26

In [60]:
X = torch.zeros(len(indices),max_length,len(vocab))

In [61]:
X.shape

torch.Size([100, 26, 749])

In [62]:
for i in range(len(indices)):
    X[[i],range(len(indices[i])),indices[i]]=1

In [63]:
Y = torch.LongTensor([labels_index[i] for i in labels])

In [64]:
class Sentiment(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(len(vocab),100,batch_first=True)
        self.classifier = nn.Linear(100,2)
        
    def forward(self,x,h):
        output,hidden = self.rnn(x,h)
        return self.classifier(hidden.view(-1,100))

In [65]:
model = Sentiment()

In [66]:
crit = nn.CrossEntropyLoss()

In [67]:
optimizer = optim.SGD(model.parameters(),lr=0.1)

In [68]:
bs=16

In [69]:
for epoch in range(2):
    for i in range(0,X.shape[0],bs):
        xb = X[i:i+bs]
        yb = Y[i:i+bs]
        h = torch.zeros(1,xb.shape[0],100)
        
        optimizer.zero_grad()
        preds = model(xb,h)
        loss = crit(preds,yb)
        loss.backward()
        optimizer.step()
        print(loss.item())
        

0.7059726119041443
0.6927284002304077
0.687088131904602
0.6846276521682739
0.7352521419525146
0.6896708011627197
0.6700661182403564
0.7321271300315857
0.7024848461151123
0.7004063129425049
0.6880125403404236
0.7133035659790039
0.6875213384628296
0.6610767841339111


## Test

In [70]:
t = test['Message'].apply(remove_html)

t = t.apply(remove_punctuation)

t = t.apply(lambda x: tokenizer.tokenize(x.lower()))

t = t.apply(remove_stopwords)

t = t.apply(word_lemmatizer)

t = t.apply(word_stemmer)

In [71]:
indices = [[vocab[token] if token in vocab else vocab['unk'] for token in doc] for doc in t]

In [72]:
max_length = len(max(t,key=lambda x:len(x)))
max_length

26

In [73]:
TY = torch.LongTensor([labels_index[i] for i in test.Category])

In [74]:
predictions=[]
for index_num in range(len(test)):

    T = torch.zeros(len(indices[index_num]),len(vocab))
    T[range(len(indices[index_num])),indices[index_num]]=1
    T = T.expand((1,-1,len(vocab)))

    h = torch.zeros(1,T.shape[0],100)
    with torch.no_grad():
        predictions.append(F.softmax(model(T,h)).argmax().item())

In [75]:
predictions = torch.LongTensor(predictions)

In [76]:
predictions

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1])

In [77]:
"Accuracy:",(predictions==TY).float().mean()

('Accuracy:', tensor(0.5000))

In [78]:
"Number of positives predicted",(predictions==1).sum()

('Number of positives predicted', tensor(100))

In [79]:
"Number of negatives predicted",(predictions==0).sum()

('Number of negatives predicted', tensor(0))

## Use nn.Embedding

In [80]:
t = train['Message'].apply(remove_html)

t = t.apply(remove_punctuation)

t = t.apply(lambda x: tokenizer.tokenize(x.lower()))

t = t.apply(remove_stopwords)

t = t.apply(word_lemmatizer)

t = t.apply(word_stemmer)

In [81]:
indices = [[vocab[token] if token in vocab else vocab['unk'] for token in doc] for doc in t]

In [82]:
max_length = len(max(t,key=lambda x:len(x)))
max_length

26

In [83]:
for i in range(len(indices)):
    if len(indices[i])<max_length:
        indices[i]+=[0]*(max_length-len(indices[i]))

In [84]:
X = torch.LongTensor(indices)
X.shape

torch.Size([100, 26])

In [85]:
Y = torch.LongTensor([labels_index[i] for i in labels])

In [88]:
class Sentiment(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(len(vocab),200,padding_idx=0)
        self.gru = nn.GRU(200,100,batch_first=True)
        self.classifier = nn.Linear(100,2)
        
    def forward(self,x,h):
        x = self.emb(x)
        output,hidden = self.gru(x,h)
        return self.classifier(hidden.view(-1,100))

In [89]:
model = Sentiment()

crit = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(),lr=0.1)

bs=32

for epoch in range(10):
    for i in range(0,X.shape[0],bs):
        xb = X[i:i+bs]
        yb = Y[i:i+bs]
        h = torch.zeros(1,xb.shape[0],100)
        
        optimizer.zero_grad()
        preds = model(xb,h)
        loss = crit(preds,yb)
        loss.backward()
        optimizer.step()
        print(loss.item())
        

0.7071463465690613
0.6979575753211975
0.6886698603630066
0.6717616319656372
0.7129810452461243
0.7022449970245361
0.681523323059082
0.6616697311401367
0.7170751690864563
0.7050150632858276
0.6769896149635315
0.655048131942749
0.7197584509849548
0.7066013813018799
0.6740244030952454
0.6506035327911377
0.7213941216468811
0.707334578037262
0.6720314025878906
0.6475527882575989
0.7222890257835388
0.7074819803237915
0.670655369758606
0.6454074382781982
0.7226772308349609
0.7072418928146362
0.6696776151657104
0.6438555717468262
0.7227279543876648
0.7067539691925049
0.6689603328704834
0.6426944732666016
0.7225596904754639
0.7061136960983276
0.6684150695800781
0.6417908668518066
0.7222533822059631
0.7053844928741455
0.6679835319519043
0.6410560011863708


## Test

In [90]:
t = test['Message'].apply(remove_html)

t = t.apply(remove_punctuation)

t = t.apply(lambda x: tokenizer.tokenize(x.lower()))

t = t.apply(remove_stopwords)

t = t.apply(word_lemmatizer)

t = t.apply(word_stemmer)

In [91]:
indices = [[vocab[token] if token in vocab else vocab['unk'] for token in doc] for doc in t]

max_length = len(max(t,key=lambda x:len(x)))
max_length

26

In [92]:
TY = torch.LongTensor([labels_index[i] for i in test.Category])

In [93]:
predictions = []
for index_num in range(len(test)):
    T = torch.LongTensor([indices[index_num]])
    h = torch.zeros(1,T.shape[0],100)
    with torch.no_grad():
        predictions.append(F.softmax(model(T,h)).argmax().item())

In [94]:
predictions = torch.LongTensor(predictions)

In [95]:
(predictions==TY).float().mean()

tensor(0.5300)

In [96]:
tp = ((TY==1)*(predictions==1)).sum().item()

tn = ((TY==0)*(predictions==0)).sum().item()

fn = ((TY==1)*(predictions==0)).sum().item()

fp = ((TY==0)*(predictions==1)).sum().item()
tp,tn,fp,tn

(49, 4, 46, 4)

#### Postive label - measures

In [97]:
print("Precision:",tp/(tp+fp))

print("Recall:",tp/(tp+fn))

Precision: 0.5157894736842106
Recall: 0.98


#### Negative label - measures

In [98]:
print("Precision:",tn/(tn+fn))

print("Recall:",tn/(tn+fp))

Precision: 0.8
Recall: 0.08


In [101]:
model.gru.weight_hh_l0.grad

tensor([[ 4.4647e-06, -2.3619e-05,  3.6112e-06,  ..., -6.4651e-06,
          3.0508e-05,  3.1949e-05],
        [ 3.8217e-06, -1.8697e-06,  2.0537e-08,  ..., -3.3056e-06,
          1.6019e-06,  1.4679e-06],
        [ 4.7258e-06,  4.9265e-06, -3.7685e-06,  ..., -6.7187e-07,
         -1.5365e-05, -1.2797e-05],
        ...,
        [ 1.3999e-04, -1.1195e-04, -2.9343e-04,  ..., -6.4626e-05,
         -4.5285e-04, -2.5619e-04],
        [ 4.0799e-04,  3.3392e-04, -1.2833e-04,  ..., -4.9071e-05,
         -8.6815e-04, -6.3166e-04],
        [-2.0011e-04, -1.0462e-04,  9.6279e-05,  ...,  3.8692e-05,
          5.1566e-04,  3.2200e-04]])

In [102]:
model.gru.weight_ih_l0.grad

tensor([[ 7.0679e-07,  1.8996e-06, -1.9252e-07,  ...,  6.4073e-06,
         -2.4909e-06,  4.2302e-06],
        [ 4.4886e-07, -9.7651e-06,  2.8628e-06,  ...,  2.8889e-06,
         -3.6571e-06,  1.9938e-06],
        [-4.1288e-06, -9.5711e-07, -3.3210e-07,  ...,  4.4093e-06,
         -7.6019e-08,  1.3912e-06],
        ...,
        [ 4.4807e-05, -1.9162e-04,  1.6100e-04,  ..., -1.8276e-05,
         -8.2698e-05,  8.5495e-05],
        [-2.9542e-04,  4.1173e-04, -1.1771e-04,  ...,  3.1162e-04,
          7.6361e-05, -2.0920e-06],
        [ 1.8940e-05,  6.3130e-05, -3.7969e-07,  ..., -7.0040e-05,
         -5.8885e-05,  3.4946e-06]])